In [1]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
#add the parent directory to the path
rpath = os.path.abspath('..')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
from src.draw import Visualize
import src.utils as utils

In [3]:
#TO csv
parent_directory = '/home/biniyam/tenx-tasks/tenxdata/'
    

df_json = SlackDataLoader.slack_parser_all(parent_directory)

df_json.to_csv('all-weeks.csv', index=False)

#TO dataframe
df = pd.read_csv('all-weeks.csv')

df.head()

/home/biniyam/tenac/tenx-tasks/src/loader.py:256: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'], unit='s')
/home/biniyam/tenac/tenx-tasks/src/loader.py:257: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  df['time_thread_start'] = pd.to_datetime(df['time_thread_start'], unit='s')
/home/biniyam/tenac/tenx-tasks/src/loader.py:258: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a fu

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,Message,I think <@U03UH397319> has to be a presenter f...,Katie Dickerson,2022-09-05 07:17:20,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
1,Message,We will be also there assisting him.,Katie Dickerson,2022-09-05 07:19:28,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
2,Message,where are you guys?,Katie Dickerson,2022-08-30 07:00:16,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
3,Message,Is the meeting at 8?,Brenda Hernandez,2022-08-30 07:40:48,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
4,Message,"I was reading, I was not there. And I thought ...",Brenda Hernandez,2022-08-30 07:40:48,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17748 entries, 0 to 17747
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   msg_type           17748 non-null  object
 1   msg_content        17724 non-null  object
 2   sender_name        17748 non-null  object
 3   msg_sent_time      17748 non-null  object
 4   msg_dist_type      17748 non-null  object
 5   time_thread_start  17748 non-null  object
 6   reply_count        17748 non-null  int64 
 7   reply_users_count  17748 non-null  int64 
 8   reply_users        17748 non-null  object
 9   tm_thread_end      17748 non-null  object
 10  channel            17748 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.5+ MB


In [5]:
df['msg_dist_type'].unique()

array(['Text', 'User', 'Link', 'Broadcast', 'Emoji', 'Rich Text Section',
       'Reshared', 'Channel'], dtype=object)

In [6]:
#convert all non-string values to string
df['msg_content'] = df['msg_content'].astype(str)

# Apply regular expressions after ensuring all values are strings
df['msg_content'] = df['msg_content'].apply(lambda x: re.sub(r'http\S+', '', x))
df['msg_content'] = df['msg_content'].apply(lambda x: re.sub(r'[^\w\s]', '', x))
df['msg_content'] = df['msg_content'].apply(lambda x: re.sub(r'\d+', '', x))
# assuming 'utils.remove_emoji' works correctly for string inputs
df['msg_content'] = df['msg_content'].apply(utils.remove_emoji)

In [7]:
df.head(10)

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,Message,I think UUH has to be a presenter for our group,Katie Dickerson,2022-09-05 07:17:20,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
1,Message,We will be also there assisting him,Katie Dickerson,2022-09-05 07:19:28,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
2,Message,where are you guys,Katie Dickerson,2022-08-30 07:00:16,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
3,Message,Is the meeting at,Brenda Hernandez,2022-08-30 07:40:48,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
4,Message,I was reading I was not there And I thought th...,Brenda Hernandez,2022-08-30 07:40:48,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
5,Message,Sorry for the confusion was in eat African time,Katie Dickerson,2022-08-30 07:49:20,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
6,Message,great job with notebooks UUH,Katie Dickerson,2022-08-30 07:49:20,Text,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
7,Message,So the time has already passed,Brenda Hernandez,2022-08-30 07:49:20,Text,2022-08-30 07:49:20,4,2,"['U03UJGRN5E0', 'U03UH397319']",2022-08-30 07:51:28,team-10
8,Message,UUJGRNE Thanks,Brenda Hernandez,2022-08-30 07:51:28,User,1970-01-01 00:00:00,0,0,[''],1970-01-01 00:00:00,team-10
9,Message,yes I have discussed wit kibatu,Katie Dickerson,2022-08-30 07:51:28,Text,2022-08-30 07:49:20,0,0,[''],1970-01-01 00:00:00,team-10


In [8]:
vectorizer = CountVectorizer(stop_words='english')
data_vectorized = vectorizer.fit_transform(df['msg_content'])

lda_model = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)



In [9]:
def display_topics(model, feature_names, no_stop_words):
    for topic_idx, topic in enumerate(model.components_):
        print ("Topic %d:" % (topic_idx + 1))
        print (" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_stop_words - 1:-1]]))
        
no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, no_top_words)

Topic 1:
working using todays want line academy getting love thought actually
Topic 2:
try just sure rolling_on_the_floor_laughing way issue nice works uugqv docker
Topic 3:
uuura uuklbr ok code uudrqh today maybe image slack uudbreg
Topic 4:
data yes does mean say model grin environment machine local
Topic 5:
think like dont install high low question submission better instance
Topic 6:
time session thank pm utc group okay hi airflow meet
Topic 7:
use link right meeting morning man let repo standup access
Topic 8:
good hello guys im make join really help think people
Topic 9:
error uugjpc uvamtfa utepyrmp check add key youre things connect
Topic 10:
did file thanks work know week challenge min need channel


In [10]:
df['msg_sent_time'] = pd.to_datetime(df['msg_sent_time'])

grouped = df.groupby(df['msg_sent_time'].dt.date)['msg_content'].apply(lambda x: ' '.join(x)).reset_index()

grouped.head()

,msg_sent_time,msg_content
0,2022-08-20,I am glad to make it to the week program I wo...
1,2022-08-21,Hi all looking forward to starting together to...
2,2022-08-22,channel Id like to welcome you all to this cha...
3,2022-08-23,TUTORIAL REMINDER channel\nPlease be aware t...
4,2022-08-24,Good morning sun_with_face channel I hope we a...


In [11]:
grouped.info()
grouped.msg_content.value_counts()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   msg_sent_time  101 non-null    object
 1   msg_content    101 non-null    object
dtypes: object(2)
memory usage: 1.7+ KB


msg_content
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

In [12]:

grouped['msg_content'][89]

'doesnt that make it  minutes to go COUNT DOWNboom\n minutes to go\nAre you ready STARTING NOW ARE YOU IN\nhere Hi all\n\nWere missing  people  \n\nHelp us get to  Hello Everyone here I have responded to the feedback form very late but I think it was a great thing to do since it have the potential of making  Academy even more interesting for the next batches Guys herewe arent trainees anymore we are a community we can make  academy even more better because we are in this together SO UUVHCVKB UUJKJGRAQ UUGNNVB  UVAMTFA   hereI post this challenge you go fill the form and come here and post I am one of them  we are only  people down so lets fix it and I start it by posting I am one of them Yeah me too me too I did fill the form  should I say I am one of them Yeah this is a post challenge just post I am one of them  UUJKJGRAQ I am one of them UVAMTFA The file the background image doesnt work Congrats to all of us and most importantly a graduation is not an end but rather a beginning\nclap

In [13]:
vectorizer = CountVectorizer(stop_words='english')
data_vectorized = vectorizer.fit_transform(grouped['msg_content'])

lda_model = LatentDirichletAllocation(n_components=10, max_iter=10, learning_method='online')
lda_Z = lda_model.fit_transform(data_vectorized)

In [23]:
no_top_words = 10
feature_names = vectorizer.get_feature_names_out()
display_topics(lda_model, feature_names, no_top_words)

Topic 1:
time just session week data yes good like dont think
Topic 2:
time think use file error just yes pm try dont
Topic 3:
time session thank good just thanks use think challenge like
Topic 4:
time session think data like yes did just good use
Topic 5:
time work session good think data dont yes just like
Topic 6:
time data think just work dont session know use yes
Topic 7:
time think use session good file yes data min just
Topic 8:
time think session good just work yes data dont hello
Topic 9:
france argentina germany croatia holland ohhhh joyjoyjoyjoyjoyjoyjoy shape vs rooney
Topic 10:
session time hello good think like yes just dont high


In [24]:
grouped.shape

(101, 2)

In [16]:
import transformers
from transformers import pipeline

ModuleNotFoundError: No module named 'transformers'